# 🧪 No-Show Prediction — Experiment Notebook

This notebook builds a **No-Show Prediction** model using **XGBoost** with **SMOTE** for handling class imbalance.

**Pipeline:**
1. Setup & Imports
2. Load Data
3. Exploratory Data Analysis (EDA)
4. Feature Engineering
5. Train/Test Split + SMOTE
6. XGBoost Model Training
7. Evaluation & Visualization
8. Save Model

---
## 1. Setup & Imports

In [ ]:
# Install dependencies (uncomment and run once if needed)
# %pip install pandas numpy scikit-learn xgboost imbalanced-learn matplotlib seaborn joblib

In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
import json
import warnings
warnings.filterwarnings("ignore")

# ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, classification_report,
    confusion_matrix, roc_curve
)

# XGBoost
from xgboost import XGBClassifier

# SMOTE for handling class imbalance
from imblearn.over_sampling import SMOTE

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", palette="muted")
print("✅ All imports loaded successfully!")

In [ ]:
# Project paths
PROJECT_ROOT = os.path.abspath(".")
DATA_PATH = os.path.join(PROJECT_ROOT, "data", "salon_bookings.csv")
MODELS_DIR = os.path.join(PROJECT_ROOT, "models")

print(f"Project Root : {PROJECT_ROOT}")
print(f"Data Path    : {DATA_PATH}")
print(f"Models Dir   : {MODELS_DIR}")

---
## 2. Load Data

In [ ]:
df = pd.read_csv(DATA_PATH)
print(f"Dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

---
## 3. Exploratory Data Analysis (EDA)

### 3.1 Target Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Count plot
target_counts = df["appointment_outcome"].value_counts()
colors = ["#2ecc71", "#e74c3c"]
axes[0].bar(target_counts.index, target_counts.values, color=colors, edgecolor="white")
axes[0].set_title("Appointment Outcome Distribution", fontweight="bold")
axes[0].set_ylabel("Count")
for i, (label, count) in enumerate(zip(target_counts.index, target_counts.values)):
    axes[0].text(i, count + 50, f"{count:,}", ha="center", fontweight="bold")

# Pie chart
axes[1].pie(target_counts.values, labels=target_counts.index, autopct="%1.1f%%",
            colors=colors, startangle=90, textprops={"fontsize": 12})
axes[1].set_title("No-Show Rate", fontweight="bold")

plt.tight_layout()
plt.show()

noshow_rate = (df['appointment_outcome'] == 'No-Show').mean()
print(f"\nNo-Show Rate: {noshow_rate:.1%}")
print(f"Class imbalance — this is why we need SMOTE!")

### 3.2 Missing Values

In [ ]:
missing = df.isnull().sum()
if missing.sum() == 0:
    print("✅ No missing values found!")
else:
    print("⚠ Missing values:")
    print(missing[missing > 0])

### 3.3 Key Feature Distributions

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Booking lead time
sns.histplot(data=df, x="booking_lead_time_days", hue="appointment_outcome",
             kde=True, ax=axes[0, 0], palette=colors)
axes[0, 0].set_title("Booking Lead Time")

# Appointment hour
sns.histplot(data=df, x="appointment_hour", hue="appointment_outcome",
             kde=True, ax=axes[0, 1], palette=colors)
axes[0, 1].set_title("Appointment Hour")

# Customer age
sns.histplot(data=df, x="customer_age", hue="appointment_outcome",
             kde=True, ax=axes[0, 2], palette=colors)
axes[0, 2].set_title("Customer Age")

# Past visit count
sns.histplot(data=df, x="past_visit_count", hue="appointment_outcome",
             kde=True, ax=axes[1, 0], palette=colors)
axes[1, 0].set_title("Past Visit Count")

# Past no-show count
sns.histplot(data=df, x="past_no_show_count", hue="appointment_outcome",
             kde=True, ax=axes[1, 1], palette=colors)
axes[1, 1].set_title("Past No-Show Count")

# Weekend vs Weekday
weekend_noshow = df.groupby(["is_weekend", "appointment_outcome"]).size().unstack(fill_value=0)
weekend_noshow.plot(kind="bar", ax=axes[1, 2], color=colors, edgecolor="white")
axes[1, 2].set_title("Weekend vs Weekday")
axes[1, 2].set_xticklabels(["Weekday", "Weekend"], rotation=0)

plt.suptitle("Feature Distributions by Appointment Outcome", fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# No-show rate by categorical features
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, col in zip(axes, ["service_type", "payment_method", "branch"]):
    noshow_rate = df.groupby(col)["appointment_outcome"].apply(
        lambda x: (x == "No-Show").mean()
    ).sort_values(ascending=False)
    noshow_rate.plot(kind="barh", ax=ax, color="#e74c3c", edgecolor="white")
    ax.set_title(f"No-Show Rate by {col}", fontweight="bold")
    ax.set_xlabel("No-Show Rate")

plt.tight_layout()
plt.show()

---
## 4. Feature Engineering

Replicating the logic from `src/feature_engineering.py` inline for full control.

In [ ]:
data = df.copy()

# ── Derived Features ────────────────────────────────────────────────────
# No-show ratio: historical no-show tendency
data["no_show_ratio"] = data["past_no_show_count"] / (data["past_visit_count"] + 1)

# Cancellation ratio: historical cancellation tendency
data["cancellation_ratio"] = data["past_cancellation_count"] / (data["past_visit_count"] + 1)

# Is new customer (first visit)
data["is_new_customer"] = (data["past_visit_count"] <= 1).astype(int)

# Is loyal customer (10+ visits)
data["is_loyal_customer"] = (data["past_visit_count"] >= 10).astype(int)

# Lead time buckets
data["lead_time_bucket"] = pd.cut(
    data["booking_lead_time_days"],
    bins=[0, 1, 3, 7, 14, 30],
    labels=["same_day", "1-3_days", "4-7_days", "1-2_weeks", "2+_weeks"],
    include_lowest=True
)

# Hour buckets
data["hour_bucket"] = pd.cut(
    data["appointment_hour"],
    bins=[8, 11, 14, 17, 21],
    labels=["morning", "midday", "afternoon", "evening"],
    include_lowest=True
)

# Age group
data["age_group"] = pd.cut(
    data["customer_age"],
    bins=[17, 25, 35, 45, 65],
    labels=["18-25", "26-35", "36-45", "46+"],
    include_lowest=True
)

print("✅ Derived features created")
print(f"   no_show_ratio, cancellation_ratio, is_new_customer, is_loyal_customer,")
print(f"   lead_time_bucket, hour_bucket, age_group")

In [ ]:
# ── Encode Categoricals ────────────────────────────────────────────────
categorical_cols = [
    "service_type", "payment_method", "branch", "day_of_week",
    "gender", "lead_time_bucket", "hour_bucket", "age_group"
]

label_encoders = {}
for col in categorical_cols:
    data[col] = data[col].astype(str)
    le = LabelEncoder()
    data[col + "_encoded"] = le.fit_transform(data[col])
    label_encoders[col] = le
    print(f"  Encoded: {col} → {len(le.classes_)} classes")

print("\n✅ All categorical features encoded")

In [ ]:
# ── Select Feature Columns ─────────────────────────────────────────────
feature_cols = [
    "booking_lead_time_days",
    "appointment_hour",
    "past_visit_count",
    "past_cancellation_count",
    "past_no_show_count",
    "is_weekend",
    "customer_age",
    "no_show_ratio",
    "cancellation_ratio",
    "is_new_customer",
    "is_loyal_customer",
    "service_type_encoded",
    "payment_method_encoded",
    "branch_encoded",
    "day_of_week_encoded",
    "gender_encoded",
    "lead_time_bucket_encoded",
    "hour_bucket_encoded",
    "age_group_encoded",
]

# Human-readable feature names for display
feature_name_map = {
    "booking_lead_time_days": "Booking Lead Time (days)",
    "appointment_hour": "Appointment Hour",
    "past_visit_count": "Past Visit Count",
    "past_cancellation_count": "Past Cancellation Count",
    "past_no_show_count": "Past No-Show Count",
    "is_weekend": "Is Weekend",
    "customer_age": "Customer Age",
    "no_show_ratio": "Historical No-Show Ratio",
    "cancellation_ratio": "Historical Cancellation Ratio",
    "is_new_customer": "Is New Customer",
    "is_loyal_customer": "Is Loyal Customer",
    "service_type_encoded": "Service Type",
    "payment_method_encoded": "Payment Method",
    "branch_encoded": "Branch",
    "day_of_week_encoded": "Day of Week",
    "gender_encoded": "Gender",
    "lead_time_bucket_encoded": "Lead Time Bucket",
    "hour_bucket_encoded": "Hour Bucket",
    "age_group_encoded": "Age Group",
}

X = data[feature_cols]
y = (data["appointment_outcome"] == "No-Show").astype(int)

print(f"✅ Feature matrix: {X.shape}")
print(f"   Target: Show={int((y == 0).sum()):,} | No-Show={int(y.sum()):,}")
X.head()

---
## 5. Train/Test Split + SMOTE

We apply **SMOTE (Synthetic Minority Over-sampling Technique)** to the **training set only** to handle the class imbalance. The test set remains untouched to give a realistic evaluation.

In [ ]:
# Split first — SMOTE is applied ONLY on training data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Before SMOTE:")
print(f"  Train set: {X_train.shape[0]:,} samples")
print(f"    Show:    {int((y_train == 0).sum()):,}")
print(f"    No-Show: {int(y_train.sum()):,}")
print(f"  Test set:  {X_test.shape[0]:,} samples")

In [ ]:
# Apply SMOTE to training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print(f"After SMOTE:")
print(f"  Train set: {X_train_smote.shape[0]:,} samples")
print(f"    Show:    {int((y_train_smote == 0).sum()):,}")
print(f"    No-Show: {int(y_train_smote.sum()):,}")
print(f"\n✅ Classes are now balanced!")

In [ ]:
# Visualize before vs after SMOTE
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Before SMOTE
before_counts = [int((y_train == 0).sum()), int(y_train.sum())]
axes[0].bar(["Show", "No-Show"], before_counts, color=["#2ecc71", "#e74c3c"], edgecolor="white")
axes[0].set_title("Before SMOTE", fontweight="bold")
axes[0].set_ylabel("Count")
for i, count in enumerate(before_counts):
    axes[0].text(i, count + 30, f"{count:,}", ha="center", fontweight="bold")

# After SMOTE
after_counts = [int((y_train_smote == 0).sum()), int(y_train_smote.sum())]
axes[1].bar(["Show", "No-Show"], after_counts, color=["#2ecc71", "#e74c3c"], edgecolor="white")
axes[1].set_title("After SMOTE", fontweight="bold")
axes[1].set_ylabel("Count")
for i, count in enumerate(after_counts):
    axes[1].text(i, count + 30, f"{count:,}", ha="center", fontweight="bold")

plt.suptitle("SMOTE: Class Balancing on Training Data", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

---
## 6. XGBoost Model Training

In [ ]:
# XGBoost hyperparameters
xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric="logloss",
    verbosity=0,
    # Note: We do NOT use scale_pos_weight since SMOTE already balances the classes
)

print("🔄 Training XGBoost on SMOTE-resampled data...")
xgb_model.fit(X_train_smote, y_train_smote)
print("✅ XGBoost training complete!")

In [ ]:
# Predictions on the ORIGINAL (untouched) test set
y_proba = xgb_model.predict_proba(X_test)[:, 1]

# Custom threshold (0.8) — only flag as No-Show when model is 80%+ confident
THRESHOLD = 0.8
y_pred = (y_proba >= THRESHOLD).astype(int)

print(f"✅ Predictions generated with threshold = {THRESHOLD}")
print(f"   Predicted No-Shows: {y_pred.sum()} out of {len(y_pred)} test samples")

---
## 7. Evaluation & Visualization

### 7.1 Metrics Summary

In [ ]:
metrics = {
    "Accuracy": accuracy_score(y_test, y_pred),
    "Precision": precision_score(y_test, y_pred),
    "Recall": recall_score(y_test, y_pred),
    "F1 Score": f1_score(y_test, y_pred),
    "AUC-ROC": roc_auc_score(y_test, y_proba),
}

print("=" * 50)
print(f"  XGBoost + SMOTE — Threshold = {THRESHOLD}")
print("=" * 50)
for name, value in metrics.items():
    bar = "█" * int(value * 30)
    print(f"  {name:12s}: {value:.4f}  {bar}")
print("=" * 50)

### 7.2 Classification Report

In [ ]:
print(f"Classification Report — XGBoost + SMOTE (Threshold = {THRESHOLD})")
print("=" * 60)
print(classification_report(y_test, y_pred, target_names=["Show", "No-Show"]))

### 7.3 Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax,
            xticklabels=["Show", "No-Show"],
            yticklabels=["Show", "No-Show"],
            annot_kws={"size": 16})
ax.set_title(f"Confusion Matrix — XGBoost + SMOTE (Threshold = {THRESHOLD})", fontsize=14, fontweight="bold")
ax.set_ylabel("Actual", fontsize=12)
ax.set_xlabel("Predicted", fontsize=12)
plt.tight_layout()
plt.show()

print(f"\nTrue Positives (correct No-Show):  {cm[1][1]}")
print(f"False Positives (false alarm):      {cm[0][1]}")
print(f"True Negatives (correct Show):      {cm[0][0]}")
print(f"False Negatives (missed No-Show):   {cm[1][0]}")

### 7.4 ROC Curve

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
auc_score = roc_auc_score(y_test, y_proba)

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(fpr, tpr, label=f"XGBoost + SMOTE (AUC = {auc_score:.4f})", color="#e74c3c", linewidth=2.5)
ax.plot([0, 1], [0, 1], "k--", alpha=0.5, label="Random (AUC = 0.5)")
ax.fill_between(fpr, tpr, alpha=0.1, color="#e74c3c")
ax.set_xlabel("False Positive Rate", fontsize=12)
ax.set_ylabel("True Positive Rate", fontsize=12)
ax.set_title("ROC Curve — XGBoost + SMOTE", fontsize=14, fontweight="bold")
ax.legend(fontsize=11)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 7.5 Feature Importance

In [ ]:
fi_df = pd.DataFrame({
    "feature": X.columns,
    "display_name": [feature_name_map.get(f, f) for f in X.columns],
    "importance": xgb_model.feature_importances_
}).sort_values("importance", ascending=True)

fig, ax = plt.subplots(figsize=(10, 8))
bar_colors = ["#3498db"] * len(fi_df)
# Highlight top 5 features in red
for i in range(-5, 0):
    bar_colors[i] = "#e74c3c"

ax.barh(fi_df["display_name"], fi_df["importance"], color=bar_colors, edgecolor="white")
ax.set_xlabel("Importance", fontsize=12)
ax.set_title("Feature Importance — XGBoost + SMOTE", fontsize=14, fontweight="bold")
ax.grid(axis="x", alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 Top 10 Features:")
for _, row in fi_df.sort_values("importance", ascending=False).head(10).iterrows():
    bar = "█" * int(row["importance"] * 100)
    print(f"  {row['display_name']:35s} {row['importance']:.4f} {bar}")

### 7.6 Prediction Probability Distribution

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ax.hist(y_proba[y_test == 0], bins=50, alpha=0.7, label="Actual Show", color="#2ecc71", edgecolor="white")
ax.hist(y_proba[y_test == 1], bins=50, alpha=0.7, label="Actual No-Show", color="#e74c3c", edgecolor="white")
ax.axvline(x=THRESHOLD, color="black", linestyle="--", linewidth=1.5, label=f"Decision Threshold ({THRESHOLD})")
ax.set_xlabel("Predicted No-Show Probability", fontsize=12)
ax.set_ylabel("Count", fontsize=12)
ax.set_title(f"Prediction Probability Distribution (Threshold = {THRESHOLD})", fontsize=14, fontweight="bold")
ax.legend(fontsize=11)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 7.7 Threshold Comparison

In [ ]:
# Compare different thresholds to understand the trade-off
print(f"{'Threshold':<12} {'Recall':<10} {'Precision':<12} {'F1':<10} {'Accuracy':<10}")
print("=" * 54)
for t in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    y_pred_t = (y_proba >= t).astype(int)
    r = recall_score(y_test, y_pred_t)
    p = precision_score(y_test, y_pred_t, zero_division=0)
    f = f1_score(y_test, y_pred_t)
    a = accuracy_score(y_test, y_pred_t)
    marker = " ◄── current" if t == THRESHOLD else ""
    print(f"  {t:<10} {r:<10.4f} {p:<12.4f} {f:<10.4f} {a:<10.4f}{marker}")

---
## 8. Save Model

In [ ]:
os.makedirs(MODELS_DIR, exist_ok=True)

# Save XGBoost model
model_path = os.path.join(MODELS_DIR, "best_model.pkl")
joblib.dump(xgb_model, model_path)
print(f"✅ Model saved:      {model_path}")

# Save label encoders
encoders_path = os.path.join(MODELS_DIR, "label_encoders.pkl")
joblib.dump(label_encoders, encoders_path)
print(f"✅ Encoders saved:   {encoders_path}")

# Save model comparison (single model row)
results_df = pd.DataFrame({"XGBoost_SMOTE": metrics}).T
results_path = os.path.join(MODELS_DIR, "model_comparison.csv")
results_df.to_csv(results_path)
print(f"✅ Comparison saved: {results_path}")

# Save feature importances
fi_save = pd.DataFrame({
    "feature": X.columns,
    "feature_display": [feature_name_map.get(f, f) for f in X.columns],
    "importance": xgb_model.feature_importances_
}).sort_values("importance", ascending=False)
fi_path = os.path.join(MODELS_DIR, "feature_importance.csv")
fi_save.to_csv(fi_path, index=False)
print(f"✅ Feature imp saved: {fi_path}")

# Save metadata
metadata = {
    "best_model_name": "XGBoost_SMOTE",
    "threshold": THRESHOLD,
    "metrics": metrics,
    "n_features": X.shape[1],
    "n_train_original": len(X_train),
    "n_train_after_smote": len(X_train_smote),
    "n_test": len(X_test),
    "feature_columns": list(X.columns),
    "smote_applied": True,
}
metadata_path = os.path.join(MODELS_DIR, "model_metadata.json")
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=2)
print(f"✅ Metadata saved:   {metadata_path}")

# Save confusion matrix
cm_path = os.path.join(MODELS_DIR, "confusion_matrix.csv")
pd.DataFrame(cm, index=["Actual Show", "Actual No-Show"], columns=["Pred Show", "Pred No-Show"]).to_csv(cm_path)
print(f"✅ Confusion matrix: {cm_path}")

print("\n" + "=" * 50)
print(f"  🎉 Pipeline complete! (Threshold = {THRESHOLD})")
print("=" * 50)

---
## 🧪 Experiment Space

Use the cells below to try new ideas — different hyperparameters, SMOTE variants, etc.

In [ ]:
# Try your experiments here!
#
# Example 1: Tune XGBoost hyperparameters with GridSearchCV
# from sklearn.model_selection import GridSearchCV
# param_grid = {
#     "n_estimators": [200, 300, 500],
#     "max_depth": [4, 6, 8],
#     "learning_rate": [0.01, 0.05, 0.1],
# }
# grid = GridSearchCV(XGBClassifier(random_state=42, eval_metric="logloss", verbosity=0),
#                     param_grid, cv=5, scoring="roc_auc", n_jobs=-1, verbose=1)
# grid.fit(X_train_smote, y_train_smote)
# print(f"Best params: {grid.best_params_}")
# print(f"Best AUC-ROC: {grid.best_score_:.4f}")
#
# Example 2: Try different SMOTE variants
# from imblearn.over_sampling import BorderlineSMOTE, ADASYN
# smote_bl = BorderlineSMOTE(random_state=42)
# X_train_bl, y_train_bl = smote_bl.fit_resample(X_train, y_train)